In [ ]:
import pandas as pd
import json
from typing import List, Dict
import re
from urllib.parse import urlparse, parse_qs, unquote
from datetime import datetime

class LogToDetailedConverter:
    """JSON 로그를 다양한 Excel 형태로 변환하는 클래스"""
    def __init__(self):
        self.input_data = []
        self.detailed_data = []
        self.detailed_deduplicated_data = []
        self.combined_data = []
        self.deduplicated_data = []

    def convert_log_to_all_formats(self, input_log_path: str, 
                                 detailed_output_path: str, 
                                 detailed_deduplicated_output_path: str,
                                 combined_output_path: str,
                                 combined_deduplicated_output_path: str):
        """
        JSON 로그 파일을 상세 형태와 결합 형태 두 가지로 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 결합 형태 중복 제거
            self._convert_to_combined_deduplicated()
            
            # 5. 상세 형태 중복 제거
            self._convert_to_detailed_deduplicated()
            
            # 6. 네 형태 모두 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_detailed_deduplicated_excel(detailed_deduplicated_output_path)
            self._save_combined_excel(combined_output_path)
            self._save_combined_deduplicated_excel(combined_deduplicated_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def convert_log_to_both_formats(self, input_log_path: str, 
                                  detailed_output_path: str, 
                                  combined_output_path: str):
        """
        기존 호환성을 위한 메서드 - 중복 제거 없이 두 가지 형태만 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행, log_time 포함)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열, log_time 포함)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 두 형태 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_combined_excel(combined_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def _load_log_file(self, log_path: str):
        """JSON 로그 파일 로드 및 파싱 (오류 자동 수정 기능 포함)"""
        try:
            self.input_data = []
            json_errors = []
            fixed_count = 0
            
            with open(log_path, 'r', encoding='utf-8') as file:
                for line_no, line in enumerate(file, 1):
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        # JSON 파싱 (새로운 형식: 직접 key-value 객체)
                        log_entry = json.loads(line)
                        
                        # 새로운 형식 처리: 직접 key-value 객체로 제공됨
                        parsed_data = self._process_new_log_format(log_entry)
                        
                        if parsed_data:
                            # 기본 로그 정보 추가
                            parsed_data['log_line'] = line_no
                            
                            self.input_data.append(parsed_data)
                    
                    except json.JSONDecodeError as e:
                        json_errors.append((line_no, str(e), line))
                        
                        # JSON 수정 시도
                        fixed_line = self._try_fix_json(line)
                        if fixed_line:
                            try:
                                log_entry = json.loads(fixed_line)
                                
                                # 새로운 형식 처리
                                parsed_data = self._process_new_log_format(log_entry)
                                
                                if parsed_data:
                                    # 기본 로그 정보 추가
                                    parsed_data['log_line'] = line_no
                                    
                                    self.input_data.append(parsed_data)
                                    fixed_count += 1
                                    print(f"✅ JSON 수정 성공 (라인 {line_no})")
                            except:
                                # 추가 수정 시도
                                fixed_line2 = self._force_fix_json(line)
                                if fixed_line2:
                                    try:
                                        log_entry = json.loads(fixed_line2)
                                        
                                        # 새로운 형식 처리
                                        parsed_data = self._process_new_log_format(log_entry)
                                        
                                        if parsed_data:
                                            # 기본 로그 정보 추가
                                            parsed_data['log_line'] = line_no
                                            
                                            self.input_data.append(parsed_data)
                                            fixed_count += 1
                                            print(f"✅ 강제 JSON 수정 성공 (라인 {line_no})")
                                    except:
                                        print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                                else:
                                    print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                        else:
                            print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
            
            # 오류 요약 출력
            if json_errors:
                print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
                if fixed_count > 0:
                    print(f"✅ 수정 성공: {fixed_count}개")
                print(f"✅ 최종 파싱된 로그: {len(self.input_data)}개")
            else:
                print(f"✅ 모든 JSON 파싱 성공")
            
            print(f"✅ 로그 파일 로드 완료: {len(self.input_data)}개 로그")
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_path}")
            raise
        except Exception as e:
            print(f"❌ 파일 로드 오류: {e}")
            raise

    def _process_new_log_format(self, log_entry: dict) -> dict:
        """새로운 로그 형식 처리: 직접 key-value 객체, 빈 값 제외, 특정 필드 제외"""
        # 제외할 필드들 (사용자 요청)
        exclude_fields = {'client_ip', 'os_version', 'platform_type', 'device_model', 'device_maker', 'event_dt', 'nth_pcid', 'nth_sid', 'user_seq'}
        
        # 처리된 데이터
        processed_data = {}
        
        # event_dttm을 logtime으로 변환 (기존 코드 호환성을 위해)
        if 'event_dttm' in log_entry:
            processed_data['logtime'] = log_entry['event_dttm']
        
        # 각 key-value 쌍 처리
        for key, value in log_entry.items():
            # 제외 필드는 건너뛰기
            if key in exclude_fields:
                continue
            
            # event_dttm은 이미 logtime으로 처리했으므로 건너뛰기
            if key == 'event_dttm':
                continue
            
            # 빈 문자열("")인 값은 제외
            if value == "":
                continue
            
            # 유효한 key-value 쌍만 포함
            processed_data[key] = value
        
        return processed_data

    def _try_fix_json(self, line: str) -> str:
        """JSON 수정 시도 - 1차 수정"""
        try:
            # 1. 마지막 쉼표 제거
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # 2. 값 내부의 따옴표를 작은따옴표로 변경
            import re
            
            # 모든 "key":"value with quotes" 패턴을 찾아서 수정
            def fix_quotes_in_value(match):
                key = match.group(1)
                value = match.group(2)
                # 값 내부의 모든 따옴표를 작은따옴표로 변경
                fixed_value = value.replace('"', "'")
                return f'"{key}":"{fixed_value}"'
            
            # 값에 따옴표가 포함된 패턴 찾기
            problem_pattern = r'"([^"]+)":"([^"]*"[^"]*)"'
            
            # 여러 번 적용하여 모든 문제 해결
            prev_line = ""
            iterations = 0
            while line != prev_line and iterations < 5:
                prev_line = line
                line = re.sub(problem_pattern, fix_quotes_in_value, line)
                iterations += 1
            
            return line
            
        except Exception:
            return None

    def _force_fix_json(self, line: str) -> str:
        """JSON 강제 수정 - 2차 수정"""
        try:
            # 기본 정리
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # JSON 구조 분석하여 키-값 쌍별로 처리
            if not (line.startswith('{') and line.endswith('}')):
                return None
            
            # 중괄호 제거
            content = line[1:-1]
            
            # 키-값 쌍들을 분리
            pairs = []
            current_pair = ""
            bracket_count = 0
            in_quotes = False
            escape_next = False
            
            for char in content:
                if escape_next:
                    current_pair += char
                    escape_next = False
                    continue
                
                if char == '\\':
                    current_pair += char
                    escape_next = True
                    continue
                
                if char == '"' and not escape_next:
                    in_quotes = not in_quotes
                
                if not in_quotes:
                    if char in '{}[]':
                        bracket_count += 1 if char in '{[' else -1
                    elif char == ',' and bracket_count == 0:
                        pairs.append(current_pair.strip())
                        current_pair = ""
                        continue
                
                current_pair += char
            
            if current_pair.strip():
                pairs.append(current_pair.strip())
            
            # 각 키-값 쌍을 수정
            fixed_pairs = []
            for pair in pairs:
                if ':' in pair:
                    try:
                        # 키와 값 분리 (첫 번째 콜론 기준)
                        colon_pos = pair.find(':')
                        key_part = pair[:colon_pos].strip()
                        value_part = pair[colon_pos+1:].strip()
                        
                        # 값 부분에서 따옴표로 둘러싸인 문자열인지 확인
                        if value_part.startswith('"') and value_part.endswith('"'):
                            # 값 내부의 따옴표를 작은따옴표로 변경
                            inner_value = value_part[1:-1]  # 양쪽 따옴표 제거
                            fixed_inner = inner_value.replace('"', "'")
                            fixed_value = f'"{fixed_inner}"'
                            fixed_pairs.append(f'{key_part}:{fixed_value}')
                        else:
                            fixed_pairs.append(pair)
                    except:
                        fixed_pairs.append(pair)
                else:
                    fixed_pairs.append(pair)
            
            # 다시 조합
            result = '{' + ','.join(fixed_pairs) + '}'
            return result
            
        except Exception:
            return None

    def _parse_request_url(self, request_url: str) -> Dict:
        """REQUEST URL에서 파라미터들을 추출 (개선된 한글 처리)"""
        if not request_url:
            return {}
        
        try:
            # 유니코드 이스케이프 디코딩
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed_url = urlparse(url)
            
            # 쿼리 파라미터를 수동으로 파싱 (복잡한 값 처리)
            query = parsed_url.query
            processed_params = {}
            
            if not query:
                return processed_params
            
            # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
            params = []
            current_param = ""
            i = 0
            
            while i < len(query):
                if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
                    if current_param:
                        params.append(current_param)
                    current_param = ""
                else:
                    current_param += query[i]
                i += 1
            
            if current_param:
                params.append(current_param)
            
            # 각 파라미터를 키=값으로 분할
            for param in params:
                if '=' not in param:
                    continue
                    
                # 첫 번째 = 기준으로만 분할 (값에 =이 포함될 수 있음)
                key, value = param.split('=', 1)
                
                # 키와 값 디코딩
                decoded_key = self._fix_korean_text(key)
                decoded_value = self._fix_korean_text(value)
                
                processed_params[decoded_key] = decoded_value
            
            return processed_params
            
        except Exception as e:
            print(f"URL 파싱 오류: {e}")
            # 실패 시 기존 방식으로 fallback
            try:
                url = request_url.encode('utf-8').decode('unicode_escape')
                parsed_url = urlparse(url)
                query_params = parse_qs(parsed_url.query, keep_blank_values=True)
                
                processed_params = {}
                for key, value_list in query_params.items():
                    raw_value = value_list[0] if value_list else ''
                    decoded_value = self._fix_korean_text(raw_value)
                    processed_params[key] = decoded_value
                
                return processed_params
            except:
                return {}

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            # URL 디코딩 먼저 시도
            from urllib.parse import unquote
            decoded = unquote(text, encoding='utf-8')
            
            # 깨진 한글 패턴이 있는지 확인
            if self._has_broken_korean(decoded):
                # Latin-1로 인코딩 후 UTF-8로 디코딩 (일반적인 mojibake 해결법)
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    pass
                    
                # 그래도 안되면 바이트 단위로 처리
                try:
                    # 문자열의 각 문자를 바이트로 변환 후 UTF-8 디코딩
                    byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                    fixed = byte_data.decode('utf-8', errors='ignore')
                    return fixed
                except:
                    pass
                    
                # UTF-8 바이트 패턴으로 복구 시도
                try:
                    # 특정 깨진 패턴들을 직접 매핑
                    broken_patterns = {
                        'ê·¹ììëª¨': '극장용모',
                        'í¥ê¸°ë¡­ê²': '향기롭게',
                        'ì¤ê¸°ì¼ì´': '수기일이',
                        'ììëª¨': '용모',
                        'ê·¹': '극'
                    }
                    
                    result = decoded
                    for broken, fixed in broken_patterns.items():
                        result = result.replace(broken, fixed)
                    
                    if result != decoded:
                        return result
                        
                except:
                    pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
            'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _format_logtime(self, logtime_str: str) -> str:
        """LOGTIME을 yyyy-mm-dd HH:MM:SS 형식으로 변환 (기존 형식과 새 형식 모두 지원)"""
        try:
            if not logtime_str:
                return ''
            
            # 새로운 형식: "2025-06-05T15:04:00.041" (ISO 8601)
            if 'T' in logtime_str:
                try:
                    # ISO 8601 형식 파싱
                    if '.' in logtime_str:
                        # 마이크로초 포함된 경우
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    else:
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    
                    # yyyy-mm-dd HH:MM:SS 형식으로 변환
                    formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                    return formatted_time
                except:
                    # ISO 형식 파싱 실패 시 기존 방식으로 fallback
                    pass
            
            # 기존 형식: "04/Jun/2025:21:46:25 +0900"
            # 월 이름을 숫자로 매핑
            month_map = {
                'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
            }
            
            # 정규식으로 파싱: "04/Jun/2025:21:46:25 +0900"
            pattern = r'(\d{2})/(\w{3})/(\d{4}):(\d{2}):(\d{2}):(\d{2})\s*\+\d{4}'
            match = re.match(pattern, logtime_str)
            
            if match:
                day, month_name, year, hour, minute, second = match.groups()
                month = month_map.get(month_name, '01')
                
                # yyyy-mm-dd HH:MM:SS 형식으로 변환
                formatted_time = f"{year}-{month}-{day} {hour}:{minute}:{second}"
                return formatted_time
            else:
                # 파싱 실패 시 원본 반환
                return logtime_str
                
        except Exception as e:
            print(f"⚠️ LOGTIME 변환 오류: {e} - 원본: {logtime_str}")
            return logtime_str

    def _convert_to_detailed(self):
        """로그 데이터를 상세 key-value 행으로 변환"""
        self.detailed_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # LOGTIME 포맷 변환
            raw_logtime = log_entry.get('logtime', '')
            formatted_logtime = self._format_logtime(raw_logtime)
            
            # 나머지 파라미터들을 key-value 쌍으로 처리
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                # 기본 필드들도 key-value 쌍에 포함 (사용자 요청)
                # page_id, click_type, act_type도 이제 key-value로 표시됨
                
                self.detailed_data.append({
                    'no': log_no,
                    'log_time': formatted_logtime,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 데이터 변환 완료: {len(self.detailed_data)}개 key-value 쌍")

    def _convert_to_combined(self):
        """로그 데이터를 결합 형태로 변환 (key, value가 쉼표로 구분된 열)"""
        self.combined_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status', 
                         'page_id', 'click_type', 'act_type'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')  # click_text 추가
            
            # 나머지 파라미터들 수집 (이제 click_text도 포함됨)
            keys = []
            values = []
            
            for key, value in log_entry.items():
                if key not in exclude_fields:
                    keys.append(key)
                    values.append(str(value) if value is not None else '')
            
            # 키와 값을 쉼표로 구분된 문자열로 결합
            keys_combined = ', '.join(keys) if keys else ''
            values_combined = ', '.join(values) if values else ''
            
            combined_entry = {
                'no': log_no,
                'page_id': page_id,
                'click_type': click_type,
                'act_type': act_type,
                'keys_combined': keys_combined,
                'values_combined': values_combined,
                'key_count': len(keys)
            }
            
            # click_text가 있으면 추가
            if click_text:
                combined_entry['click_text'] = click_text
            
            self.combined_data.append(combined_entry)
        
        print(f"✅ 결합 데이터 변환 완료: {len(self.combined_data)}개 로그")

    def _convert_to_combined_deduplicated(self):
        """결합 데이터에서 중복을 제거한 형태로 변환 (key_count가 큰 것 우선)"""
        if not self.combined_data:
            print("⚠️ 결합 데이터가 없어 중복 제거를 수행할 수 없습니다.")
            return
        
        # pandas DataFrame으로 변환
        df = pd.DataFrame(self.combined_data)
        
        # 중복 제거 전 개수
        original_count = len(df)
        
        # page_id, click_type, act_type을 기준으로 중복 제거
        # click_text가 있는 경우 추가로 포함
        dedup_columns = ['page_id', 'click_type', 'act_type']
        
        # click_text 컬럼이 존재하고 비어있지 않은 행이 있으면 추가
        if 'click_text' in df.columns and df['click_text'].notna().any() and (df['click_text'] != '').any():
            dedup_columns.append('click_text')
        
        # key_count 내림차순으로 정렬 (큰 값이 먼저 오도록)
        df_sorted = df.sort_values(['key_count'] + dedup_columns, ascending=[False] + [True] * len(dedup_columns))
        
        # 중복 제거 (key_count가 큰 것이 먼저 오므로 keep='first'로 큰 값이 남음)
        df_deduplicated = df_sorted.drop_duplicates(subset=dedup_columns, keep='first')
        
        # 원래 순서대로 재정렬 (no 기준)
        df_deduplicated = df_deduplicated.sort_values('no')
        
        # 중복 제거 후 개수
        deduplicated_count = len(df_deduplicated)
        removed_count = original_count - deduplicated_count
        
        # DataFrame을 다시 딕셔너리 리스트로 변환
        self.deduplicated_data = df_deduplicated.to_dict('records')
        
        print(f"✅ 결합 형태 중복 제거 완료 (key_count 큰 것 우선):")
        print(f"   원본: {original_count}개")
        print(f"   중복 제거 후: {deduplicated_count}개")
        print(f"   제거된 중복: {removed_count}개")
        print(f"   중복 제거 기준: {', '.join(dedup_columns)} + key_count 큰 것 우선")

    def _convert_to_detailed_deduplicated(self):
        """상세 데이터에서 중복 제거한 형태로 변환 (결합 형태 중복 제거된 log_no 기준으로 필터링)"""
        if not self.deduplicated_data:
            print("⚠️ 결합 형태 중복 제거 데이터가 없어 상세 중복 제거를 수행할 수 없습니다.")
            return
        
        # 결합 형태에서 중복 제거된 log_no들 추출
        deduplicated_log_nos = set(item['no'] for item in self.deduplicated_data)
        
        print(f"✅ 결합 형태 중복 제거에서 유효한 log_no: {len(deduplicated_log_nos)}개")
        
        self.detailed_deduplicated_data = []
        
        # 제외할 기본 필드들 (메타데이터) - _convert_to_detailed와 동일
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 결합 형태에서 중복 제거된 log_no만 처리
            if log_no not in deduplicated_log_nos:
                continue
            
            # 기본 정보 추출 - _convert_to_detailed와 동일
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # LOGTIME 포맷 변환 - _convert_to_detailed와 동일
            raw_logtime = log_entry.get('logtime', '')
            formatted_logtime = self._format_logtime(raw_logtime)
            
            # 나머지 파라미터들을 key-value 쌍으로 처리 - _convert_to_detailed와 동일
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                # 기본 필드들도 key-value 쌍에 포함 (사용자 요청)
                # page_id, click_type, act_type도 이제 key-value로 표시됨
                
                self.detailed_deduplicated_data.append({
                    'no': log_no,
                    'log_time': formatted_logtime,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 형태 중복 제거 완료 (결합 형태 기준 필터링):")
        print(f"   필터링된 key-value 쌍: {len(self.detailed_deduplicated_data)}개")
        print(f"   기준: 결합 형태 중복 제거된 {len(deduplicated_log_nos)}개 로그")

    def _extract_click_text_from_keys(self, row):
        """keys_combined에서 click_text 값을 추출하려고 시도"""
        try:
            keys = row.get('keys_combined', '').split(', ')
            values = row.get('values_combined', '').split(', ')
            
            if len(keys) == len(values):
                for i, key in enumerate(keys):
                    if key.strip() == 'click_text':
                        return values[i].strip() if i < len(values) else ''
            return ''
        except:
            return ''

    def _save_detailed_excel(self, output_path: str):
        """상세 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'act_type', 'click_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_data)}개 key-value 쌍")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_detailed_deduplicated_excel(self, output_path: str):
        """상세 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_deduplicated_data)}개 key-value 쌍 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_deduplicated_excel(self, output_path: str):
        """결합 형태 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(5, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 형태 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.deduplicated_data)}개 로그 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_excel(self, output_path: str):
        """결합 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.combined_data)
            
            if df.empty:
                print("⚠️ 저장할 결합 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(5, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.combined_data)}개 로그")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def get_conversion_summary(self) -> Dict:
        """변환 결과 요약 정보 반환"""
        if not self.detailed_data:
            return {"error": "변환된 데이터가 없습니다."}
        
        # 통계 계산
        total_logs = len(self.input_data)
        total_key_values = len(self.detailed_data)
        avg_keys_per_log = total_key_values / total_logs if total_logs > 0 else 0
        
        # 가장 많은 키를 가진 로그 찾기
        key_counts = {}
        for item in self.detailed_data:
            log_no = item['no']
            key_counts[log_no] = key_counts.get(log_no, 0) + 1
        
        max_keys = max(key_counts.values()) if key_counts else 0
        min_keys = min(key_counts.values()) if key_counts else 0
        
        # 고유한 키 수 계산
        unique_keys = set(item['key'] for item in self.detailed_data)
        
        # 중복 제거 통계
        combined_count = len(self.combined_data) if self.combined_data else 0
        combined_deduplicated_count = len(self.deduplicated_data) if self.deduplicated_data else 0
        combined_duplicates_removed = combined_count - combined_deduplicated_count
        
        detailed_count = len(self.detailed_data) if self.detailed_data else 0
        detailed_deduplicated_count = len(self.detailed_deduplicated_data) if self.detailed_deduplicated_data else 0
        detailed_duplicates_removed = detailed_count - detailed_deduplicated_count
        
        return {
            "total_logs": total_logs,
            "total_key_value_pairs": total_key_values,
            "average_keys_per_log": round(avg_keys_per_log, 2),
            "max_keys_in_single_log": max_keys,
            "min_keys_in_single_log": min_keys,
            "unique_keys_count": len(unique_keys),
            "unique_keys": sorted(list(unique_keys)),
            "combined_logs_count": combined_count,
            "combined_deduplicated_count": combined_deduplicated_count,
            "combined_duplicates_removed": combined_duplicates_removed,
            "detailed_key_value_pairs": detailed_count,
            "detailed_deduplicated_count": detailed_deduplicated_count,
            "detailed_duplicates_removed": detailed_duplicates_removed
        }

    def preview_data(self, num_rows: int = 10):
        """변환된 상세 데이터 미리보기"""
        print(f"\n📋 상세 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':15} | {'Value':20}")
        print("-" * 160)
        
        if self.detailed_data:
            for item in self.detailed_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:15] if len(item['key']) > 15 else item['key']
                value = item['value'][:20] if len(item['value']) > 20 else item['value']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:15} | {key:15} | {value:20}")
            
            if len(self.detailed_data) > num_rows:
                print(f"... 외 {len(self.detailed_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 결합 형태 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':15} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':20} | {'Values':20} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        
        print("-" * 170)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:15 if has_click_text else 20] if len(item['page_id']) > (15 if has_click_text else 20) else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:15} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:20} | {values:20} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:25} | {values:25} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_detailed_deduplicated_data(self, num_rows: int = 10):
        """상세 중복 제거된 데이터 미리보기"""
        print(f"\n📋 상세 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':15} | {'Value':20}")
        print("-" * 160)
        
        if self.detailed_deduplicated_data:
            for item in self.detailed_deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:15] if len(item['key']) > 15 else item['key']
                value = item['value'][:20] if len(item['value']) > 20 else item['value']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:15} | {key:15} | {value:20}")
            
            if len(self.detailed_deduplicated_data) > num_rows:
                print(f"... 외 {len(self.detailed_deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':15} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':20} | {'Values':20} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        
        print("-" * 170)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:15 if has_click_text else 20] if len(item['page_id']) > (15 if has_click_text else 20) else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:15} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:20} | {values:20} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:20} | {values:20} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_data(self, num_rows: int = 5):
        """결합 데이터 미리보기"""
        print(f"\n📋 결합 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':30} | {'Values':30} | {'Count':5}")
        print("-" * 170)
        
        if self.combined_data:
            for item in self.combined_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                keys = item['keys_combined'][:30] if len(item['keys_combined']) > 30 else item['keys_combined']
                values = item['values_combined'][:30] if len(item['values_combined']) > 30 else item['values_combined']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:30} | {values:30} | {item['key_count']:5}")
            
            if len(self.combined_data) > num_rows:
                print(f"... 외 {len(self.combined_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def get_unique_keys_analysis(self):
        """고유 키 분석"""
        if not self.detailed_data:
            print("분석할 데이터가 없습니다.")
            return
        
        # 키별 빈도수 계산
        key_frequency = {}
        for item in self.detailed_data:
            key = item['key']
            key_frequency[key] = key_frequency.get(key, 0) + 1
        
        # 빈도수 순으로 정렬
        sorted_keys = sorted(key_frequency.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n📊 키 분석 (총 {len(sorted_keys)}개 고유 키):")
        print("-" * 50)
        print(f"{'Key':30} | {'빈도수':10}")
        print("-" * 50)
        
        for key, freq in sorted_keys[:20]:  # 상위 20개만 표시
            key_display = key[:30] if len(key) > 30 else key
            print(f"{key_display:30} | {freq:10}")
        
        if len(sorted_keys) > 20:
            print(f"... 외 {len(sorted_keys) - 20}개 키")


def run_log_converter_example():
    """로그 변환기 사용 예시"""
    converter = LogToDetailedConverter()

    print("=== JSON 로그를 Excel 형태로 변환 (JSON 오류 자동 수정 + 중복 제거 기능 포함) ===\n")
    
    # 로그 파일을 네 가지 형태로 변환
    converter.convert_log_to_all_formats(
        "log_file.txt",                              # 입력 JSON 로그 파일
        "./result/log_analysis_detailed.xlsx",                # 상세 형태 출력
        "./result/log_analysis_detailed_deduplicated.xlsx",   # 상세 형태 중복 제거 출력
        import pandas as pd
import json
from typing import List, Dict
import re
from urllib.parse import urlparse, parse_qs, unquote
from datetime import datetime

class LogToDetailedConverter:
    """JSON 로그를 다양한 Excel 형태로 변환하는 클래스"""
    def __init__(self):
        self.input_data = []
        self.detailed_data = []
        self.detailed_deduplicated_data = []
        self.combined_data = []
        self.deduplicated_data = []

    def convert_log_to_all_formats(self, input_log_path: str, 
                                 detailed_output_path: str, 
                                 detailed_deduplicated_output_path: str,
                                 combined_output_path: str,
                                 combined_deduplicated_output_path: str):
        """
        JSON 로그 파일을 상세 형태와 결합 형태 두 가지로 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 결합 형태 중복 제거
            self._convert_to_combined_deduplicated()
            
            # 5. 상세 형태 중복 제거
            self._convert_to_detailed_deduplicated()
            
            # 6. 네 형태 모두 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_detailed_deduplicated_excel(detailed_deduplicated_output_path)
            self._save_combined_excel(combined_output_path)
            self._save_combined_deduplicated_excel(combined_deduplicated_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def convert_log_to_both_formats(self, input_log_path: str, 
                                  detailed_output_path: str, 
                                  combined_output_path: str):
        """
        기존 호환성을 위한 메서드 - 중복 제거 없이 두 가지 형태만 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행, log_time 포함)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열, log_time 포함)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 두 형태 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_combined_excel(combined_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def _load_log_file(self, log_path: str):
        """JSON 로그 파일 로드 및 파싱 (오류 자동 수정 기능 포함)"""
        try:
            self.input_data = []
            json_errors = []
            fixed_count = 0
            
            with open(log_path, 'r', encoding='utf-8') as file:
                for line_no, line in enumerate(file, 1):
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        # JSON 파싱 (새로운 형식: 직접 key-value 객체)
                        log_entry = json.loads(line)
                        
                        # 새로운 형식 처리: 직접 key-value 객체로 제공됨
                        parsed_data = self._process_new_log_format(log_entry)
                        
                        if parsed_data:
                            # 기본 로그 정보 추가
                            parsed_data['log_line'] = line_no
                            
                            self.input_data.append(parsed_data)
                    
                    except json.JSONDecodeError as e:
                        json_errors.append((line_no, str(e), line))
                        
                        # JSON 수정 시도
                        fixed_line = self._try_fix_json(line)
                        if fixed_line:
                            try:
                                log_entry = json.loads(fixed_line)
                                
                                # 새로운 형식 처리
                                parsed_data = self._process_new_log_format(log_entry)
                                
                                if parsed_data:
                                    # 기본 로그 정보 추가
                                    parsed_data['log_line'] = line_no
                                    
                                    self.input_data.append(parsed_data)
                                    fixed_count += 1
                                    print(f"✅ JSON 수정 성공 (라인 {line_no})")
                            except:
                                # 추가 수정 시도
                                fixed_line2 = self._force_fix_json(line)
                                if fixed_line2:
                                    try:
                                        log_entry = json.loads(fixed_line2)
                                        
                                        # 새로운 형식 처리
                                        parsed_data = self._process_new_log_format(log_entry)
                                        
                                        if parsed_data:
                                            # 기본 로그 정보 추가
                                            parsed_data['log_line'] = line_no
                                            
                                            self.input_data.append(parsed_data)
                                            fixed_count += 1
                                            print(f"✅ 강제 JSON 수정 성공 (라인 {line_no})")
                                    except:
                                        print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                                else:
                                    print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                        else:
                            print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
            
            # 오류 요약 출력
            if json_errors:
                print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
                if fixed_count > 0:
                    print(f"✅ 수정 성공: {fixed_count}개")
                print(f"✅ 최종 파싱된 로그: {len(self.input_data)}개")
            else:
                print(f"✅ 모든 JSON 파싱 성공")
            
            print(f"✅ 로그 파일 로드 완료: {len(self.input_data)}개 로그")
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_path}")
            raise
        except Exception as e:
            print(f"❌ 파일 로드 오류: {e}")
            raise

    def _process_new_log_format(self, log_entry: dict) -> dict:
        """새로운 로그 형식 처리: 직접 key-value 객체, 빈 값 제외, 특정 필드 제외"""
        # 제외할 필드들 (사용자 요청)
        exclude_fields = {'client_ip', 'os_version', 'platform_type', 'device_model', 'device_maker', 'event_dt', 'nth_pcid', 'nth_sid', 'user_seq'}
        
        # 처리된 데이터
        processed_data = {}
        
        # event_dttm을 logtime으로 변환 (기존 코드 호환성을 위해)
        if 'event_dttm' in log_entry:
            processed_data['logtime'] = log_entry['event_dttm']
        
        # 각 key-value 쌍 처리
        for key, value in log_entry.items():
            # 제외 필드는 건너뛰기
            if key in exclude_fields:
                continue
            
            # event_dttm은 이미 logtime으로 처리했으므로 건너뛰기
            if key == 'event_dttm':
                continue
            
            # 빈 문자열("")인 값은 제외
            if value == "":
                continue
            
            # 유효한 key-value 쌍만 포함
            processed_data[key] = value
        
        return processed_data

    def _try_fix_json(self, line: str) -> str:
        """JSON 수정 시도 - 1차 수정"""
        try:
            # 1. 마지막 쉼표 제거
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # 2. 값 내부의 따옴표를 작은따옴표로 변경
            import re
            
            # 모든 "key":"value with quotes" 패턴을 찾아서 수정
            def fix_quotes_in_value(match):
                key = match.group(1)
                value = match.group(2)
                # 값 내부의 모든 따옴표를 작은따옴표로 변경
                fixed_value = value.replace('"', "'")
                return f'"{key}":"{fixed_value}"'
            
            # 값에 따옴표가 포함된 패턴 찾기
            problem_pattern = r'"([^"]+)":"([^"]*"[^"]*)"'
            
            # 여러 번 적용하여 모든 문제 해결
            prev_line = ""
            iterations = 0
            while line != prev_line and iterations < 5:
                prev_line = line
                line = re.sub(problem_pattern, fix_quotes_in_value, line)
                iterations += 1
            
            return line
            
        except Exception:
            return None

    def _force_fix_json(self, line: str) -> str:
        """JSON 강제 수정 - 2차 수정"""
        try:
            # 기본 정리
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # JSON 구조 분석하여 키-값 쌍별로 처리
            if not (line.startswith('{') and line.endswith('}')):
                return None
            
            # 중괄호 제거
            content = line[1:-1]
            
            # 키-값 쌍들을 분리
            pairs = []
            current_pair = ""
            bracket_count = 0
            in_quotes = False
            escape_next = False
            
            for char in content:
                if escape_next:
                    current_pair += char
                    escape_next = False
                    continue
                
                if char == '\\':
                    current_pair += char
                    escape_next = True
                    continue
                
                if char == '"' and not escape_next:
                    in_quotes = not in_quotes
                
                if not in_quotes:
                    if char in '{}[]':
                        bracket_count += 1 if char in '{[' else -1
                    elif char == ',' and bracket_count == 0:
                        pairs.append(current_pair.strip())
                        current_pair = ""
                        continue
                
                current_pair += char
            
            if current_pair.strip():
                pairs.append(current_pair.strip())
            
            # 각 키-값 쌍을 수정
            fixed_pairs = []
            for pair in pairs:
                if ':' in pair:
                    try:
                        # 키와 값 분리 (첫 번째 콜론 기준)
                        colon_pos = pair.find(':')
                        key_part = pair[:colon_pos].strip()
                        value_part = pair[colon_pos+1:].strip()
                        
                        # 값 부분에서 따옴표로 둘러싸인 문자열인지 확인
                        if value_part.startswith('"') and value_part.endswith('"'):
                            # 값 내부의 따옴표를 작은따옴표로 변경
                            inner_value = value_part[1:-1]  # 양쪽 따옴표 제거
                            fixed_inner = inner_value.replace('"', "'")
                            fixed_value = f'"{fixed_inner}"'
                            fixed_pairs.append(f'{key_part}:{fixed_value}')
                        else:
                            fixed_pairs.append(pair)
                    except:
                        fixed_pairs.append(pair)
                else:
                    fixed_pairs.append(pair)
            
            # 다시 조합
            result = '{' + ','.join(fixed_pairs) + '}'
            return result
            
        except Exception:
            return None

    def _parse_request_url(self, request_url: str) -> Dict:
        """REQUEST URL에서 파라미터들을 추출 (개선된 한글 처리)"""
        if not request_url:
            return {}
        
        try:
            # 유니코드 이스케이프 디코딩
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed_url = urlparse(url)
            
            # 쿼리 파라미터를 수동으로 파싱 (복잡한 값 처리)
            query = parsed_url.query
            processed_params = {}
            
            if not query:
                return processed_params
            
            # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
            params = []
            current_param = ""
            i = 0
            
            while i < len(query):
                if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
                    if current_param:
                        params.append(current_param)
                    current_param = ""
                else:
                    current_param += query[i]
                i += 1
            
            if current_param:
                params.append(current_param)
            
            # 각 파라미터를 키=값으로 분할
            for param in params:
                if '=' not in param:
                    continue
                    
                # 첫 번째 = 기준으로만 분할 (값에 =이 포함될 수 있음)
                key, value = param.split('=', 1)
                
                # 키와 값 디코딩
                decoded_key = self._fix_korean_text(key)
                decoded_value = self._fix_korean_text(value)
                
                processed_params[decoded_key] = decoded_value
            
            return processed_params
            
        except Exception as e:
            print(f"URL 파싱 오류: {e}")
            # 실패 시 기존 방식으로 fallback
            try:
                url = request_url.encode('utf-8').decode('unicode_escape')
                parsed_url = urlparse(url)
                query_params = parse_qs(parsed_url.query, keep_blank_values=True)
                
                processed_params = {}
                for key, value_list in query_params.items():
                    raw_value = value_list[0] if value_list else ''
                    decoded_value = self._fix_korean_text(raw_value)
                    processed_params[key] = decoded_value
                
                return processed_params
            except:
                return {}

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            # URL 디코딩 먼저 시도
            from urllib.parse import unquote
            decoded = unquote(text, encoding='utf-8')
            
            # 깨진 한글 패턴이 있는지 확인
            if self._has_broken_korean(decoded):
                # Latin-1로 인코딩 후 UTF-8로 디코딩 (일반적인 mojibake 해결법)
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    pass
                    
                # 그래도 안되면 바이트 단위로 처리
                try:
                    # 문자열의 각 문자를 바이트로 변환 후 UTF-8 디코딩
                    byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                    fixed = byte_data.decode('utf-8', errors='ignore')
                    return fixed
                except:
                    pass
                    
                # UTF-8 바이트 패턴으로 복구 시도
                try:
                    # 특정 깨진 패턴들을 직접 매핑
                    broken_patterns = {
                        'ê·¹ììëª¨': '극장용모',
                        'í¥ê¸°ë¡­ê²': '향기롭게',
                        'ì¤ê¸°ì¼ì´': '수기일이',
                        'ììëª¨': '용모',
                        'ê·¹': '극'
                    }
                    
                    result = decoded
                    for broken, fixed in broken_patterns.items():
                        result = result.replace(broken, fixed)
                    
                    if result != decoded:
                        return result
                        
                except:
                    pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
            'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _format_logtime(self, logtime_str: str) -> str:
        """LOGTIME을 yyyy-mm-dd HH:MM:SS 형식으로 변환 (기존 형식과 새 형식 모두 지원)"""
        try:
            if not logtime_str:
                return ''
            
            # 새로운 형식: "2025-06-05T15:04:00.041" (ISO 8601)
            if 'T' in logtime_str:
                try:
                    # ISO 8601 형식 파싱
                    if '.' in logtime_str:
                        # 마이크로초 포함된 경우
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    else:
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    
                    # yyyy-mm-dd HH:MM:SS 형식으로 변환
                    formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                    return formatted_time
                except:
                    # ISO 형식 파싱 실패 시 기존 방식으로 fallback
                    pass
            
            # 기존 형식: "04/Jun/2025:21:46:25 +0900"
            # 월 이름을 숫자로 매핑
            month_map = {
                'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
            }
            
            # 정규식으로 파싱: "04/Jun/2025:21:46:25 +0900"
            pattern = r'(\d{2})/(\w{3})/(\d{4}):(\d{2}):(\d{2}):(\d{2})\s*\+\d{4}'
            match = re.match(pattern, logtime_str)
            
            if match:
                day, month_name, year, hour, minute, second = match.groups()
                month = month_map.get(month_name, '01')
                
                # yyyy-mm-dd HH:MM:SS 형식으로 변환
                formatted_time = f"{year}-{month}-{day} {hour}:{minute}:{second}"
                return formatted_time
            else:
                # 파싱 실패 시 원본 반환
                return logtime_str
                
        except Exception as e:
            print(f"⚠️ LOGTIME 변환 오류: {e} - 원본: {logtime_str}")
            return logtime_str

    def _convert_to_detailed(self):
        """로그 데이터를 상세 key-value 행으로 변환"""
        self.detailed_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # LOGTIME 포맷 변환
            raw_logtime = log_entry.get('logtime', '')
            formatted_logtime = self._format_logtime(raw_logtime)
            
            # 나머지 파라미터들을 key-value 쌍으로 처리
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                # 기본 필드들도 key-value 쌍에 포함 (사용자 요청)
                # page_id, click_type, act_type도 이제 key-value로 표시됨
                
                self.detailed_data.append({
                    'no': log_no,
                    'log_time': formatted_logtime,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 데이터 변환 완료: {len(self.detailed_data)}개 key-value 쌍")

    def _convert_to_combined(self):
        """로그 데이터를 결합 형태로 변환 (key, value가 쉼표로 구분된 열)"""
        self.combined_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status', 
                         'page_id', 'click_type', 'act_type'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')  # click_text 추가
            
            # 나머지 파라미터들 수집 (이제 click_text도 포함됨)
            keys = []
            values = []
            
            for key, value in log_entry.items():
                if key not in exclude_fields:
                    keys.append(key)
                    values.append(str(value) if value is not None else '')
            
            # 키와 값을 쉼표로 구분된 문자열로 결합
            keys_combined = ', '.join(keys) if keys else ''
            values_combined = ', '.join(values) if values else ''
            
            combined_entry = {
                'no': log_no,
                'page_id': page_id,
                'click_type': click_type,
                'act_type': act_type,
                'keys_combined': keys_combined,
                'values_combined': values_combined,
                'key_count': len(keys)
            }
            
            # click_text가 있으면 추가
            if click_text:
                combined_entry['click_text'] = click_text
            
            self.combined_data.append(combined_entry)
        
        print(f"✅ 결합 데이터 변환 완료: {len(self.combined_data)}개 로그")

    def _convert_to_combined_deduplicated(self):
        """결합 데이터에서 중복을 제거한 형태로 변환 (key_count가 큰 것 우선)"""
        if not self.combined_data:
            print("⚠️ 결합 데이터가 없어 중복 제거를 수행할 수 없습니다.")
            return
        
        # pandas DataFrame으로 변환
        df = pd.DataFrame(self.combined_data)
        
        # 중복 제거 전 개수
        original_count = len(df)
        
        # page_id, click_type, act_type을 기준으로 중복 제거
        # click_text가 있는 경우 추가로 포함
        dedup_columns = ['page_id', 'click_type', 'act_type']
        
        # click_text 컬럼이 존재하고 비어있지 않은 행이 있으면 추가
        if 'click_text' in df.columns and df['click_text'].notna().any() and (df['click_text'] != '').any():
            dedup_columns.append('click_text')
        
        # key_count 내림차순으로 정렬 (큰 값이 먼저 오도록)
        df_sorted = df.sort_values(['key_count'] + dedup_columns, ascending=[False] + [True] * len(dedup_columns))
        
        # 중복 제거 (key_count가 큰 것이 먼저 오므로 keep='first'로 큰 값이 남음)
        df_deduplicated = df_sorted.drop_duplicates(subset=dedup_columns, keep='first')
        
        # 원래 순서대로 재정렬 (no 기준)
        df_deduplicated = df_deduplicated.sort_values('no')
        
        # 중복 제거 후 개수
        deduplicated_count = len(df_deduplicated)
        removed_count = original_count - deduplicated_count
        
        # DataFrame을 다시 딕셔너리 리스트로 변환
        self.deduplicated_data = df_deduplicated.to_dict('records')
        
        print(f"✅ 결합 형태 중복 제거 완료 (key_count 큰 것 우선):")
        print(f"   원본: {original_count}개")
        print(f"   중복 제거 후: {deduplicated_count}개")
        print(f"   제거된 중복: {removed_count}개")
        print(f"   중복 제거 기준: {', '.join(dedup_columns)} + key_count 큰 것 우선")

    def _convert_to_detailed_deduplicated(self):
        """상세 데이터에서 중복 제거한 형태로 변환 (결합 형태 중복 제거된 log_no 기준으로 필터링)"""
        if not self.deduplicated_data:
            print("⚠️ 결합 형태 중복 제거 데이터가 없어 상세 중복 제거를 수행할 수 없습니다.")
            return
        
        # 결합 형태에서 중복 제거된 log_no들 추출
        deduplicated_log_nos = set(item['no'] for item in self.deduplicated_data)
        
        print(f"✅ 결합 형태 중복 제거에서 유효한 log_no: {len(deduplicated_log_nos)}개")
        
        self.detailed_deduplicated_data = []
        
        # 제외할 기본 필드들 (메타데이터) - _convert_to_detailed와 동일
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 결합 형태에서 중복 제거된 log_no만 처리
            if log_no not in deduplicated_log_nos:
                continue
            
            # 기본 정보 추출 - _convert_to_detailed와 동일
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # LOGTIME 포맷 변환 - _convert_to_detailed와 동일
            raw_logtime = log_entry.get('logtime', '')
            formatted_logtime = self._format_logtime(raw_logtime)
            
            # 나머지 파라미터들을 key-value 쌍으로 처리 - _convert_to_detailed와 동일
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                # 기본 필드들도 key-value 쌍에 포함 (사용자 요청)
                # page_id, click_type, act_type도 이제 key-value로 표시됨
                
                self.detailed_deduplicated_data.append({
                    'no': log_no,
                    'log_time': formatted_logtime,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 형태 중복 제거 완료 (결합 형태 기준 필터링):")
        print(f"   필터링된 key-value 쌍: {len(self.detailed_deduplicated_data)}개")
        print(f"   기준: 결합 형태 중복 제거된 {len(deduplicated_log_nos)}개 로그")

    def _extract_click_text_from_keys(self, row):
        """keys_combined에서 click_text 값을 추출하려고 시도"""
        try:
            keys = row.get('keys_combined', '').split(', ')
            values = row.get('values_combined', '').split(', ')
            
            if len(keys) == len(values):
                for i, key in enumerate(keys):
                    if key.strip() == 'click_text':
                        return values[i].strip() if i < len(values) else ''
            return ''
        except:
            return ''

    def _save_detailed_excel(self, output_path: str):
        """상세 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'act_type', 'click_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_data)}개 key-value 쌍")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_detailed_deduplicated_excel(self, output_path: str):
        """상세 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_deduplicated_data)}개 key-value 쌍 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_deduplicated_excel(self, output_path: str):
        """결합 형태 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(5, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 형태 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.deduplicated_data)}개 로그 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_excel(self, output_path: str):
        """결합 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.combined_data)
            
            if df.empty:
                print("⚠️ 저장할 결합 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의 (log_time을 두 번째 열로)
            column_order = ['no', 'log_time', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(5, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.combined_data)}개 로그")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def get_conversion_summary(self) -> Dict:
        """변환 결과 요약 정보 반환"""
        if not self.detailed_data:
            return {"error": "변환된 데이터가 없습니다."}
        
        # 통계 계산
        total_logs = len(self.input_data)
        total_key_values = len(self.detailed_data)
        avg_keys_per_log = total_key_values / total_logs if total_logs > 0 else 0
        
        # 가장 많은 키를 가진 로그 찾기
        key_counts = {}
        for item in self.detailed_data:
            log_no = item['no']
            key_counts[log_no] = key_counts.get(log_no, 0) + 1
        
        max_keys = max(key_counts.values()) if key_counts else 0
        min_keys = min(key_counts.values()) if key_counts else 0
        
        # 고유한 키 수 계산
        unique_keys = set(item['key'] for item in self.detailed_data)
        
        # 중복 제거 통계
        combined_count = len(self.combined_data) if self.combined_data else 0
        combined_deduplicated_count = len(self.deduplicated_data) if self.deduplicated_data else 0
        combined_duplicates_removed = combined_count - combined_deduplicated_count
        
        detailed_count = len(self.detailed_data) if self.detailed_data else 0
        detailed_deduplicated_count = len(self.detailed_deduplicated_data) if self.detailed_deduplicated_data else 0
        detailed_duplicates_removed = detailed_count - detailed_deduplicated_count
        
        return {
            "total_logs": total_logs,
            "total_key_value_pairs": total_key_values,
            "average_keys_per_log": round(avg_keys_per_log, 2),
            "max_keys_in_single_log": max_keys,
            "min_keys_in_single_log": min_keys,
            "unique_keys_count": len(unique_keys),
            "unique_keys": sorted(list(unique_keys)),
            "combined_logs_count": combined_count,
            "combined_deduplicated_count": combined_deduplicated_count,
            "combined_duplicates_removed": combined_duplicates_removed,
            "detailed_key_value_pairs": detailed_count,
            "detailed_deduplicated_count": detailed_deduplicated_count,
            "detailed_duplicates_removed": detailed_duplicates_removed
        }

    def preview_data(self, num_rows: int = 10):
        """변환된 상세 데이터 미리보기"""
        print(f"\n📋 상세 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':15} | {'Value':20}")
        print("-" * 160)
        
        if self.detailed_data:
            for item in self.detailed_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:15] if len(item['key']) > 15 else item['key']
                value = item['value'][:20] if len(item['value']) > 20 else item['value']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:15} | {key:15} | {value:20}")
            
            if len(self.detailed_data) > num_rows:
                print(f"... 외 {len(self.detailed_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 결합 형태 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':15} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':20} | {'Values':20} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        
        print("-" * 170)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:15 if has_click_text else 20] if len(item['page_id']) > (15 if has_click_text else 20) else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:15} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:20} | {values:20} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:25} | {values:25} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_detailed_deduplicated_data(self, num_rows: int = 10):
        """상세 중복 제거된 데이터 미리보기"""
        print(f"\n📋 상세 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':15} | {'Value':20}")
        print("-" * 160)
        
        if self.detailed_deduplicated_data:
            for item in self.detailed_deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:15] if len(item['key']) > 15 else item['key']
                value = item['value'][:20] if len(item['value']) > 20 else item['value']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:15} | {key:15} | {value:20}")
            
            if len(self.detailed_deduplicated_data) > num_rows:
                print(f"... 외 {len(self.detailed_deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':15} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':20} | {'Values':20} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        
        print("-" * 170)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:15 if has_click_text else 20] if len(item['page_id']) > (15 if has_click_text else 20) else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:15} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:20} | {values:20} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:20] if len(item['keys_combined']) > 20 else item['keys_combined']
                    values = item['values_combined'][:20] if len(item['values_combined']) > 20 else item['values_combined']
                    print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:20} | {values:20} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_data(self, num_rows: int = 5):
        """결합 데이터 미리보기"""
        print(f"\n📋 결합 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 170)
        print(f"{'No':3} | {'Log Time':19} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Keys':30} | {'Values':30} | {'Count':5}")
        print("-" * 170)
        
        if self.combined_data:
            for item in self.combined_data[:num_rows]:
                log_time = item.get('log_time', '')[:19] if len(item.get('log_time', '')) > 19 else item.get('log_time', '')
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                keys = item['keys_combined'][:30] if len(item['keys_combined']) > 30 else item['keys_combined']
                values = item['values_combined'][:30] if len(item['values_combined']) > 30 else item['values_combined']
                
                print(f"{item['no']:3} | {log_time:19} | {page_id:20} | {click_type:12} | {act_type:12} | {keys:30} | {values:30} | {item['key_count']:5}")
            
            if len(self.combined_data) > num_rows:
                print(f"... 외 {len(self.combined_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def get_unique_keys_analysis(self):
        """고유 키 분석"""
        if not self.detailed_data:
            print("분석할 데이터가 없습니다.")
            return
        
        # 키별 빈도수 계산
        key_frequency = {}
        for item in self.detailed_data:
            key = item['key']
            key_frequency[key] = key_frequency.get(key, 0) + 1
        
        # 빈도수 순으로 정렬
        sorted_keys = sorted(key_frequency.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n📊 키 분석 (총 {len(sorted_keys)}개 고유 키):")
        print("-" * 50)
        print(f"{'Key':30} | {'빈도수':10}")
        print("-" * 50)
        
        for key, freq in sorted_keys[:20]:  # 상위 20개만 표시
            key_display = key[:30] if len(key) > 30 else key
            print(f"{key_display:30} | {freq:10}")
        
        if len(sorted_keys) > 20:
            print(f"... 외 {len(sorted_keys) - 20}개 키")


def run_log_converter_example():
    """로그 변환기 사용 예시"""
    converter = LogToDetailedConverter()

    print("=== JSON 로그를 Excel 형태로 변환 (JSON 오류 자동 수정 + 중복 제거 기능 포함) ===\n")
    
    # 로그 파일을 네 가지 형태로 변환
    converter.convert_log_to_all_formats(
        "log_file.txt",                              # 입력 JSON 로그 파일
        "./result/log_analysis_detailed.xlsx",                # 상세 형태 출력
        "./result/log_analysis_detailed_deduplicated.xlsx",   # 상세 형태 중복 제거 출력
        "./result/log_analysis_combined.xlsx",                # 결합 형태 출력
        "./result/log_analysis_combined_deduplicated.xlsx"    # 결합 형태 중복 제거 출력
    )
    
    # 결과 요약 출력
    summary = converter.get_conversion_summary()
    print(f"\n📊 변환 결과 요약:")
    for key, value in summary.items():
        if key != 'unique_keys':  # unique_keys는 너무 길어서 별도 출력
            print(f"   {key}: {value}")
    
    # 데이터 미리보기
    converter.preview_data(10)
    converter.preview_detailed_deduplicated_data(10)
    converter.preview_combined_data(5)
    converter.preview_combined_deduplicated_data(5)
    
    # 키 분석
    converter.get_unique_keys_analysis()


if __name__ == "__main__":
    run_log_converter_example()               # 결합 형태 출력
        "./result/log_analysis_combined_deduplicated.xlsx"    # 결합 형태 중복 제거 출력
    )
    
    # 결과 요약 출력
    summary = converter.get_conversion_summary()
    print(f"\n📊 변환 결과 요약:")
    for key, value in summary.items():
        if key != 'unique_keys':  # unique_keys는 너무 길어서 별도 출력
            print(f"   {key}: {value}")
    
    # 데이터 미리보기
    converter.preview_data(10)
    converter.preview_detailed_deduplicated_data(10)
    converter.preview_combined_data(5)
    converter.preview_combined_deduplicated_data(5)
    
    # 키 분석
    converter.get_unique_keys_analysis()


if __name__ == "__main__":
    run_log_converter_example()

=== JSON 로그를 Excel 형태로 변환 (JSON 오류 자동 수정 + 중복 제거 기능 포함) ===

✅ 모든 JSON 파싱 성공
✅ 로그 파일 로드 완료: 14개 로그
✅ 상세 데이터 변환 완료: 153개 key-value 쌍
✅ 결합 데이터 변환 완료: 14개 로그
✅ 결합 형태 중복 제거 완료 (key_count 큰 것 우선):
   원본: 14개
   중복 제거 후: 7개
   제거된 중복: 7개
   중복 제거 기준: page_id, click_type, act_type, click_text + key_count 큰 것 우선
✅ 결합 형태 중복 제거에서 유효한 log_no: 7개
✅ 상세 형태 중복 제거 완료 (결합 형태 기준 필터링):
   필터링된 key-value 쌍: 80개
   기준: 결합 형태 중복 제거된 7개 로그
✅ 상세 Excel 파일 저장: ./result/log_analysis_detailed.xlsx
   총 153개 key-value 쌍
✅ 상세 중복 제거 Excel 파일 저장: ./result/log_analysis_detailed_deduplicated.xlsx
   총 80개 key-value 쌍 (중복 제거됨)
✅ 결합 Excel 파일 저장: ./result/log_analysis_combined.xlsx
   총 14개 로그
✅ 결합 형태 중복 제거 Excel 파일 저장: ./result/log_analysis_combined_deduplicated.xlsx
   총 7개 로그 (중복 제거됨)

📊 변환 결과 요약:
   total_logs: 14
   total_key_value_pairs: 153
   average_keys_per_log: 10.93
   max_keys_in_single_log: 19
   min_keys_in_single_log: 5
   unique_keys_count: 25
   combined_logs_count: 14
   combined_deduplicated_count: 7
 